# 📜 Projeto Final - Capacitação IA (Ciclo 3)
# 🎓 Alunos: Filipe da Silva Rodrigues e Rodrigo Serafim Floriano da Silva

## 💻 Bibliotecas Necessárias

In [ ]:
# Instalação de bibliotecas necessárias para execução do código
%pip install numpy pandas scikit-learn mlflow xgboost lightgbm --quiet

In [ ]:
# Tratamento de Dataset e Métricas
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Modelos de Treinamento
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
# Multi-layer Perceptron (MLP)
from sklearn.neural_network import MLPClassifier
# Support Vector Machine
from sklearn.svm import SVC
# Random Forest, Bagging e Gradient Boosting
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
# XGBoost
from xgboost import XGBClassifier
# LightGBM
from lightgbm import LGBMClassifier

# Armazenamento e Análise de Modelos
import mlflow
import mlflow.sklearn

# Terminal
import warnings
from IPython.display import clear_output
warnings.filterwarnings("ignore")



---

👾 **Dataset de Classificação - Kaggle: Water Quality**

Esse dataframe é um conjunto de dados que contém informações sobre a qualidade da água e sua potabilidade. As variáveis são:

- `ph`: o valor do pH da água (0 a 14).
- `Hardness`: a capacidade da água de precipitar sabão em mg/L.
- `Solids`: sólidos totais dissolvidos em ppm.
- `Chloramines`: quantidade de cloraminas em ppm.
- `Sulfate`: quantidade de sulfatos dissolvidos em mg/L.
- `Conductivity`: condutividade elétrica da água em μS/cm.
- `Organic_carbon`: quantidade de carbono orgânico em ppm.
- `Trihalomethanes`: quantidade de trihalometanos em μg/L.
- `Turbidity`: medida da propriedade de emissão de luz da água em NTU.
- `Potability`: indica se a água é segura para consumo humano (1 = Potável, 0 = Não potável).

✅ **Objetivo:** Prever se a água é potável ou não com base nas características coletadas.

---


In [ ]:
# Carregar o dataset
url = 'water_potability.csv'
dataset = pd.read_csv(url)

# Analisar o dataset
print('\nInformações do Dataset:\n')
display(dataset.info())

print('\nVerificar Valores Nulos:\n')
display(dataset.isnull().sum())

# Exibir o dataset original
print('\nDataset Original:\n')
display(dataset)

# Criar uma cópia do dataset para efetuar os devidos tratamentos
df = dataset.copy()

# Normalizando os dados das features na escala (0..1)
columns_to_normalize = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']
df[columns_to_normalize] = MinMaxScaler().fit_transform(df[columns_to_normalize])

# Separar os dados para o tratamento de features
target = df['Potability'].copy()
features = df.drop('Potability', axis=1).copy()

# Combinando as features transformadas com o target
df = pd.concat(
    [features.reset_index(drop=True), target.reset_index(drop=True)], axis=1)

# Exibindo o DataFrame tratado com as colunas renomeadas
print('\nDataset Tratado para Treinamento:\n')
display(df)

# Análise de correlação entre as features
print('\nMatriz de Correlação:\n')
correlation_matrix = df.corr()
display(correlation_matrix)

# Separando os dados 
y = df['Potability']  # Coluna 'Potability'
x = df.drop('Potability', axis=1)  # Todas as outras colunas

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=None)


## 🧪 Experimentos no MLFLOW

In [ ]:
# Definir os modelos e suas variações de parâmetros
models = {
    "Decision Tree": [
        {"criterion": "gini", "max_depth": 10, "min_samples_split": 4},
        {"criterion": "entropy", "max_depth": 20, "min_samples_split": 10},
        {"criterion": "gini", "max_depth": 15, "min_samples_split": 5},
    ],
    "Support Vector Machine": [
        {"C": 1.0, "kernel": "linear"},
        {"C": 10.0, "kernel": "rbf"},
        {"C": 100.0, "kernel": "poly", "degree": 3},
    ],
    "Multi-layer Perceptron": [
        {"hidden_layer_sizes": (100, 50), "activation": "relu",
         "solver": "adam", "max_iter": 1000},
        {"hidden_layer_sizes": (50, 50, 50), "activation": "tanh",
         "solver": "adam", "max_iter": 1000},
        {"hidden_layer_sizes": (100, 50), "activation": "relu",
         "solver": "lbfgs", "max_iter": 1000},
    ],
    "Bagging": [
        {"n_estimators": 10, "random_state": 42},
    ],
    "Random Forest": [
        {"n_estimators": 100, "max_depth": 10, "random_state": 42},
    ],
    "Gradient Boosting": [
        {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 3, "random_state": 42},
    ],
    "XGBoost": [
        {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 3, "verbosity": 0},
    ],
    "LightGBM": [
        {"n_estimators": 100, "learning_rate": 0.1, "max_depth": -1, "verbosity": -1},
    ]
}

# Mapeamento de nomes de modelos para classes
model_classes = {
    "Decision Tree": DecisionTreeClassifier,
    "Support Vector Machine": SVC,
    "Multi-layer Perceptron": MLPClassifier,
    "Bagging": BaggingClassifier,
    "Random Forest": RandomForestClassifier,
    "Gradient Boosting": GradientBoostingClassifier,
    "XGBoost": XGBClassifier,
    "LightGBM": LGBMClassifier,
}


In [ ]:
# Configurar o caminho relativo para os artefatos
mlflow.set_tracking_uri("file:./mlruns")

# Preparar o ambiente do MLFlow e iniciar o experimento
# lista para armazenar os resultados
results = []

# Iniciar o experimento
mlflow.set_experiment("exp_projeto_ciclo_3")

# Contador para evitar conflitos de nomes
counter = 0

# Run para registrar os modelos treinados
with mlflow.start_run(run_name="Modelos Treinados") as main_run: # Principal
    for model_name, param_variations in models.items():
        for params in param_variations:
            counter += 1
            with mlflow.start_run(run_name=f"{counter}. {model_name}", nested=True): # Aninhada
                # Instanciar o modelo usando o dicionário de classes
                model = model_classes[model_name](**params)

                # Treinar o modelo
                model.fit(x_train, y_train)

                # Fazer previsões no conjunto de teste
                predictions = model.predict(x_test)

                # Calcular as métricas
                accuracy = accuracy_score(y_test, predictions)
                precision = precision_score(y_test, predictions, average='weighted')
                recall = recall_score(y_test, predictions, average='weighted')
                f1 = f1_score(y_test, predictions, average='weighted')

                # Registrar parâmetros individualmente
                for key, value in params.items():
                    mlflow.log_param(key, str(value))
                
                # Registrar métricas e modelo
                mlflow.log_metric("Accuracy", accuracy)
                mlflow.log_metric("Precision", precision)
                mlflow.log_metric("Recall", recall)
                mlflow.log_metric("F1 Score", f1)
                
                mlflow.sklearn.log_model(model, model_name)

                # Armazenar resultados
                results.append({"model": model_name, "params": params,
                               "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1})

    # Selecionar os 3 melhores modelos com base na métrica F1 Score
    best_models = sorted(results, key=lambda x: x["F1 Score"], reverse=True)[:3]

# Limpar a saída do terminal
clear_output(wait=True)

print("\nMelhores Modelos:\n")
for model_info in best_models:
    print(model_info)
print("\n\n")

# Run para registrar os melhores modelos
acc = 4
with mlflow.start_run(run_name="Melhores Modelos") as main_run: # Principal
    for model_info in reversed(best_models):
            acc -= 1
            model_name = model_info["model"]
            params = model_info["params"]
            
            with mlflow.start_run(run_name=f"{acc}. {model_name}", nested=True): # Aninhada
                # Instanciar o modelo usando o dicionário de classes
               model = model_classes[model_name](**params)
                
                # Treinar o modelo
                model.fit(x_train, y_train)

                # Fazer previsões no conjunto de teste
                predictions = model.predict(x_test)

                # Calcular as métricas
                accuracy = accuracy_score(y_test, predictions)
                precision = precision_score(y_test, predictions, average='weighted')
                recall = recall_score(y_test, predictions, average='weighted')
                f1 = f1_score(y_test, predictions, average='weighted')

                # Registrar parâmetros individualmente
                for key, value in params.items():
                    mlflow.log_param(key, str(value))

                # Registrar métricas e modelo
                mlflow.log_metric("Accuracy", accuracy)
                mlflow.log_metric("Precision", precision)
                mlflow.log_metric("Recall", recall)
                mlflow.log_metric("F1 Score", f1)

                mlflow.sklearn.log_model(
                    model, model_name, registered_model_name=model_name)


## 💾 Modelos Registrados no MLFLOW

In [ ]:
import subprocess

# Definir o tracking URI do MLflow
mlflow_tracking_uri = 'file:./mlruns'  # Caminho relativo

mlflow.set_tracking_uri(mlflow_tracking_uri)

# Iniciar o MLflow UI em um subprocesso separado
mlflow_process = subprocess.Popen(["mlflow", "ui", "--host", "127.0.0.1", "--port", "5000"])

# Exibir a URL do MLflow UI
print("MLflow UI está rodando em http://127.0.0.1:5000")

In [ ]:
# Parar o subprocesso do MLflow UI
mlflow_process.terminate()

# Confirmar que o MLflow UI foi parado
print("MLflow UI foi parado")